# three-tier-geolocation - REST API Server

## Initialization

In [1]:
import requests
import os

# Declare constants
# REST_URL = "http://127.0.0.1:5000/uploader"
REST_URL = "http://127.0.0.1:5000"

DATASETS_PATH = "."

In [2]:
REQUESTED_FILE_NAME = "results.txt" 

def use_api(dataset_1p, dataset_3p, rtt_method, geolocation_method):
    output_file = os.path.join(OUTPUT_PATH, f"results_learn={dataset_1p}_test={dataset_3p}.txt")
    
    print("start:", dataset_1p, dataset_3p)
    
    dataset_1p_path = os.path.join(DATASETS_PATH, dataset_1p)
    dataset_3p_path = os.path.join(DATASETS_PATH, dataset_3p)
    
    FILE_MEASUREMENTS_1P = os.path.join(dataset_1p_path, "measurements-1party.csv")
    FILE_MEASUREMENTS_3P = os.path.join(dataset_3p_path, "measurements-3party.csv")
    FILE_SERVERS_1P = os.path.join(dataset_1p_path, "servers-1party.csv")
    FILE_SERVERS_3P = os.path.join(dataset_3p_path, "servers-3party.csv")
    FILE_DATACENTERS = os.path.join(dataset_3p_path, "datacenters.csv")
    FILE_SOLUTION = os.path.join(dataset_3p_path, "solution.csv")

    response = requests.post(REST_URL + "/rest",
                             files=[
                                 ('measurements-1party', ('measurements-1party.csv', open(FILE_MEASUREMENTS_1P, 'rb'), 'application/octet-stream')),
                                 ('servers-1party', ('servers-1party.csv', open(FILE_SERVERS_1P, 'rb'), 'application/octet-stream')),
                                 ('measurements-3party', ('measurements-3party.csv', open(FILE_MEASUREMENTS_3P, 'rb'), 'application/octet-stream')),
                                 ('servers-3party', ('servers-3party.csv', open(FILE_SERVERS_3P, 'rb'), 'application/octet-stream')),
                                 ('datacenters', ('datacenters.csv', open(FILE_DATACENTERS, 'rb'), 'application/octet-stream')),
                                 ('solution', ('solution.csv', open(FILE_SOLUTION, 'rb'), 'application/octet-stream'))
                             ],
                             data={'rtt_method': rtt_method, 'geolocation_method': geolocation_method})

    if response.status_code != 200:
        print("Error: POST received bad response code:", response.status_code)
        return

    requested_file_url = response.json()['Assets'][REQUESTED_FILE_NAME]
    print(requested_file_url)

    second_response = requests.get(requested_file_url)

    if second_response.status_code != 200:
        print("Error: GET received bad response code:", response.status_code)
        return

    with open(output_file, 'w', encoding="utf-8") as f:
        f.writelines(second_response.text)

## Use the API

### Single request

In [12]:
rtt_method = "Optimization"  # rtt_method = "Subtraction", "Optimization"
geolocation_method = "Multilateration"  # geolocation_method = "Multilateration", "Profiling"
OUTPUT_PATH = "api-client-out/temp"  # make sure this directory exists

use_api(dataset_1p="DS-B1", dataset_3p="DS-F4", rtt_method=rtt_method, geolocation_method=geolocation_method)


start: DS-B1 DS-F4
http://127.0.0.1:5000/GetFile/tmp5b2bd7fx/output/results.txt


### Multiple requests 

In [11]:
rtt_method = "Optimization"  # rtt_method = "Subtraction", "Optimization"
geolocation_method = "Profiling"  # geolocation_method = "Multilateration", "Profiling"
OUTPUT_PATH = f"api-client-out/{rtt_method}_{geolocation_method}"  # make sure this directory exists

DATASETS = ["DS-B1", "DS-F1", "DS-F2", "DS-F3", "DS-F4", ]

for d1 in DATASETS:
    for d2 in DATASETS:
        use_api(dataset_1p=d1, dataset_3p=d2, rtt_method=rtt_method, geolocation_method=geolocation_method)

start: DS-B1 DS-B1
http://127.0.0.1:5000/GetFile/tmpujw_fdsm/output/results.txt
start: DS-B1 DS-F1


KeyboardInterrupt: 